## Prompt Engineering en LangSmith

### Importamos variables de entorno

In [1]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

### Ejecutamos un Pull del Prompt desde Prompthub

In [ ]:
from langsmith import Client
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("eli5-profesor", include_model=True)

### Setup Aplicación IA 

Primero configuremos nuestra herramienta de búsqueda web, como siempre.

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=1)

C:\Users\sergi\AppData\Local\Temp\ipykernel_55428\632376592.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(max_results=1)


Ahora creemos nuestra aplicación, igual que en el módulo de trazas. Esta vez, nuestro prompt es el que se obtuvo desde PromptHub.


In [4]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

# Creamos la aplicación
openai_client = wrap_openai(OpenAI())

@traceable
def search(question):
    web_docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in web_docs])
    return web_results
    
@traceable
def explain(question, context):
    prompt_inputs = {"question": question, "context": context}
    messages = prompt.invoke(prompt_inputs)
    messages = [{"role": messages.type, "content": messages.content}]
    # Mapea los roles a los valores válidos de OpenAI
    role_map = {"ai": "assistant", "human": "user"}
    for msg in messages:
        if msg["role"] in role_map:
            msg["role"] = role_map[msg["role"]]
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)
    return answer

### Test Application

In [5]:
question = "Qué es la macroeconomía?"
print(eli5(question))

c:\Users\sergi\Documents\langsmith\eli5\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


La macroeconomía es la rama de la economía que analiza y estudia el comportamiento de la economía en su conjunto, en lugar de enfocarse en mercados individuales o sectores específicos. Se interesa en variables agregadas como el producto interno bruto (PIB), la tasa de desempleo, la inflación y el crecimiento económico. 

Al igual que cuidar de un jardín grande, donde cada planta representa un sector de la economía, la macroeconomía busca entender cómo interactúan todos estos elementos para asegurar un crecimiento saludable y sostenible. Los líderes y responsables de la política económica utilizan la macroeconomía para tomar decisiones que pueden afectar a todos los ciudadanos, buscando mejorar el bienestar general. Por ejemplo, pueden implementar políticas fiscales o monetarias para estimular la economía o controlar la inflación, con el objetivo de crear un entorno en el que todos puedan prosperar.
